In [0]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima    

import warnings
warnings.filterwarnings("ignore")

In [0]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Covid19/week4/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Covid19/week4/test.csv')
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Covid19/week4/submission.csv')

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24414 entries, 0 to 24413
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              24414 non-null  int64  
 1   Country_Region  24414 non-null  object 
 2   Province_State  10374 non-null  object 
 3   Date            24414 non-null  object 
 4   ConfirmedCases  24414 non-null  float64
 5   Fatalities      24414 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.1+ MB


In [0]:
test.info()

In [0]:
submission.info()

In [0]:
train['Country/Province'] = np.where(train['Province_State'].isna() == False, train['Country_Region'] + '/' + train['Province_State'], train['Country_Region'])
test['Country/Province'] = np.where(test['Province_State'].isna() == False, test['Country_Region'] + '/' + test['Province_State'], test['Country_Region'])

In [0]:
countries = train['Country/Province'].unique()
submission=pd.DataFrame(columns=submission.columns)
for country in countries:
    train_df = train[train['Country/Province'] == country]
    test_df = test[test['Country/Province'] == country]

########### Farecasting ConfirmedCases...........

    X_train_conf = train_df['ConfirmedCases'].values
    p,d,q = auto_arima(X_train_conf).order
    model_conf = SARIMAX(X_train_conf,order=(p,d,q),seasonal_order=(0,0,0,0))
    results = model_conf.fit()
    fcast_conf = results.predict(len(X_train_conf)-7,len(X_train_conf)+len(test_df)-8,typ='levels')


########### Farecasting Fatalities.............

    X_train_fat = train_df['Fatalities'].values
    p,d,q = auto_arima(X_train_fat).order
    model_fat = SARIMAX(X_train_fat,order=(p,d,q),seasonal_order=(0,0,0,0))
    results = model_fat.fit()
    fcast_fat = results.predict(len(X_train_fat)-7,len(X_train_fat)+len(test_df)-8,typ='levels')

    test_id = test_df['ForecastId'].values.tolist()
    test_res = pd.DataFrame(columns=submission.columns)
    test_res['ForecastId'] = test_id
    test_res['ConfirmedCases'] = np.rint(fcast_conf)
    test_res['Fatalities'] = np.rint(fcast_fat)
    submission = submission.append(test_res)

In [0]:
submission.head(10)

In [0]:
submission.to_csv('submission.csv',index=False)